In [115]:
import pandas as pd
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

Step 1: Load the data set into a pandas dataframe. For this example I only use the red wine data set.

In [11]:
wine_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df = pd.read_csv(wine_url, sep=';')

In [24]:
# I personally find it annoying to work with column names that have spaces in them, so I renamed the columns
df = df.rename(lambda x: x.replace(' ', '_'), axis='columns')

Step 2: Do some basic EDA to get an understanding of what's in the data. I'm assuming we've covered this topic by this point and am passing over this point quickly for now. Students can expand on this with their own visualizations or analysis.

In [75]:
df.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [73]:
df.isnull().apply(sum, axis=0)

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [74]:
df.corr()['quality']

fixed_acidity           0.124052
volatile_acidity       -0.390558
citric_acid             0.226373
residual_sugar          0.013732
chlorides              -0.128907
free_sulfur_dioxide    -0.050656
total_sulfur_dioxide   -0.185100
density                -0.174919
pH                     -0.057731
sulphates               0.251397
alcohol                 0.476166
quality                 1.000000
Name: quality, dtype: float64

Step 3: Split the data into train and test. I'm assuming we've covered this topic and the students already understand why it's necessary.

In [49]:
train_data, test_data, train_labels, test_labels = train_test_split(df.drop('quality', axis = 1), df.quality,
                                                                    test_size = 0.2, random_state = 99, stratify=df.quality)

In [54]:
# I sometimes make a mistake, especially in the order of the four assignments, so let's double check the split
print('length all data', len(df))
print('length train data', len(train_data))
print('length test data', len(test_data))

length all data 1599
length train data 1279
length test data 320


In [58]:
train_data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
682,8.5,0.460,0.31,2.25,0.078,32.0,58.0,0.99800,3.33,0.54,9.8
1186,6.6,0.800,0.03,7.80,0.079,6.0,12.0,0.99630,3.52,0.50,12.2
1389,6.7,0.480,0.02,2.20,0.080,36.0,111.0,0.99524,3.10,0.53,9.7
559,13.0,0.470,0.49,4.30,0.085,6.0,47.0,1.00210,3.30,0.68,12.7
519,7.3,0.365,0.49,2.50,0.088,39.0,106.0,0.99660,3.36,0.78,11.0


In [59]:
train_labels.head()

682     5
1186    5
1389    5
559     6
519     5
Name: quality, dtype: int64

Step 4: Build a simple linear regression model to have a base to work off and make sure everything is working. Earlier in the lesson I'll introduce linear regression concepts, like how 

In [111]:
reg_model = LinearRegression()
reg_model.fit(train_data, train_labels)
test_pred = reg_model.predict(test_data)

In [102]:
# we can peak inside our model 
print(reg_model.intercept_)
print(reg_model.coef_)

40.36144283432613
[ 4.05984464e-02 -1.18031242e+00 -1.54759629e-01  1.97262259e-02
 -1.64827569e+00  4.61801020e-03 -3.43629260e-03 -3.64092578e+01
 -3.26376962e-01  8.48856310e-01  2.48672371e-01]


Step 5: Evaluate our model. Options include:

* R-squared: The proportion of variance explained by the model. The most common way to evaluate a linear regression model. Values closer to 1 are better. One issue is that it will increase as more features are added to the model. 

Other options include:
* Mean Absolute Error: The mean of the absolute values of the errors.
* Mean Squared Error: The mean of the squares of the errors.
* Root Mean Squared Error: The square root of the mean squared error.

Mean squared error is sometimes preferred over mean absolute error because it penalizes larger errors. However, root mean squared error is an even better option because it is more easily interpretable -- it has the same 'units' as the target variable.

In [103]:
print('R-squared score: ', r2_score(test_labels, test_pred))
print('Another way to get the R-squared score: ', reg_model.score(test_data, test_labels))
print('Mean Squared Error: ', mean_squared_error(test_labels, test_pred))
print('Root Mean Squared Error: ', math.sqrt(mean_squared_error(test_labels, test_pred)))

R-squared score:  0.3469259111366333
Another way to get the R-squared score:  0.3469259111366333
Mean Squared Error:  0.42141136226304515
Root Mean Squared Error:  0.6491620462280933


In [104]:
# making a root mean squared error function for later
def rmse(true_labels, pred_labels):
    return math.sqrt(mean_squared_error(true_labels, pred_labels))

Now that we have a way to compare two different models, we can iterate on our base model to get a better predictor.

The first thing we're going to do is standardize our input variables, transforming them to have a mean of 0 and a variance of 1.

In [116]:
scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.fit_transform(test_data)

reg_model = LinearRegression()
reg_model.fit(scaled_train_data, train_labels)
test_pred = reg_model.predict(scaled_test_data)

print('R2: ', r2_score(test_labels, test_pred))
print('RMSE: ', rmse(test_labels, test_pred))

R2:  0.347124763513621
RMSE:  0.6490632080977871


Scaling the data unfortunately didn't lead to much of a boost in performance. 

Another thing we could try is using only a subset of the features. Experiment with different combinations of the features, maybe only the ones that have the highest correlation, to see how that affects model performance.

To extend the lesson further, try some other types of linear models. Here I'll test the lasso and ridge regression models, both of which seek to use regularization to reduce the complexity of our models.

In [130]:
scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.fit_transform(test_data)

ridge_model = Ridge(alpha=0.1)
ridge_model.fit(scaled_train_data, train_labels)
test_pred = ridge_model.predict(scaled_test_data)

print('R2: ', r2_score(test_labels, test_pred))
print('RMSE: ', rmse(test_labels, test_pred))

R2:  0.3471241184276357
RMSE:  0.6490635287574263


In [136]:
scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.fit_transform(test_data)

lasso_model = Lasso(alpha=0.01)
lasso_model.fit(scaled_train_data, train_labels)
test_pred = lasso_model.predict(scaled_test_data)

print('R2: ', r2_score(test_labels, test_pred))
print('RMSE: ', rmse(test_labels, test_pred))

R2:  0.349211156747087
RMSE:  0.6480252726340663


In this example, our regularized models do not see much of an improvement.